In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn. datasets import load_digits
digits = load_digits()
from google.colab import files
from google.colab import drive

# Initial imports
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Mount the Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Full dataset (~4 min.)
arrest_df = pd.read_csv("gdrive/My Drive/Colab Notebooks/arrest_10yr_clean.csv")

print(arrest_df.shape)
arrest_df.head()

(2855141, 16)


,ARREST_KEY,ARREST_DATE,PD_CD,KY_CD,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,82422385,2012-01-01,759.0,359.0,PL 1950500,M,K,79,18-24,M,BLACK,997366.0,192877.0,40.696078,-73.952702,POINT (-73.95270199199997 40.69607807800003)
1,82422694,2012-01-01,101.0,344.0,PL 1200001,M,M,6,18-24,M,WHITE,983794.0,204737.0,40.728641,-74.001647,POINT (-74.00164669699996 40.72864066600005)
2,82422331,2012-01-01,905.0,347.0,VTL11920U2,M,K,72,25-44,M,WHITE HISPANIC,989519.0,178298.0,40.656070,-73.981012,POINT (-73.981011557 40.656070154000076)
3,82422817,2012-01-01,782.0,236.0,PL 2650101,M,Q,115,25-44,M,BLACK HISPANIC,1017897.0,219850.0,40.770058,-73.878527,POINT (-73.87852744499997 40.77005803100008)
4,82422890,2012-01-01,258.0,351.0,PL 1456002,M,Q,104,18-24,M,WHITE,1012109.0,195368.0,40.702881,-73.899525,POINT (-73.89952491499997 40.70288116000006)


In [3]:
arrest_df['ARREST_BORO'].value_counts()

K    794319
M    760908
B    634965
Q    558602
S    106347
Name: ARREST_BORO, dtype: int64

In [4]:
columns_drop = ["X_COORD_CD", "Y_COORD_CD", "Latitude", "Longitude", "Lon_Lat", "ARREST_KEY", "ARREST_DATE", "LAW_CODE"]
arrest_df = arrest_df.drop(columns_drop, 1)
arrest_df.head()

<ipython-input-4-7bf7e7903fc0>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  arrest_df = arrest_df.drop(columns_drop, 1)


,PD_CD,KY_CD,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE
0,759.0,359.0,M,K,79,18-24,M,BLACK
1,101.0,344.0,M,M,6,18-24,M,WHITE
2,905.0,347.0,M,K,72,25-44,M,WHITE HISPANIC
3,782.0,236.0,M,Q,115,25-44,M,BLACK HISPANIC
4,258.0,351.0,M,Q,104,18-24,M,WHITE


In [5]:
arrest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2855141 entries, 0 to 2855140
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   PD_CD            float64
 1   KY_CD            float64
 2   LAW_CAT_CD       object 
 3   ARREST_BORO      object 
 4   ARREST_PRECINCT  int64  
 5   AGE_GROUP        object 
 6   PERP_SEX         object 
 7   PERP_RACE        object 
dtypes: float64(2), int64(1), object(5)
memory usage: 174.3+ MB


In [6]:
arrest_df["AGE_GROUP"].value_counts()

25-44    1382728
18-24     707682
45-64     554451
<18       180976
65+        29304
Name: AGE_GROUP, dtype: int64

In [7]:
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("25-", "")
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("18-", "")
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("45-", "")
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("<18", "17")
arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("+", "")
arrest_df["AGE_GROUP"].value_counts()

<ipython-input-7-82071f55752f>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  arrest_df["AGE_GROUP"]= arrest_df["AGE_GROUP"].str.replace("+", "")


44    1382728
24     707682
64     554451
17     180976
65      29304
Name: AGE_GROUP, dtype: int64

In [8]:
# Encode letters to number

arrest_df_encoded = pd.get_dummies(arrest_df, columns=["PERP_SEX", "PERP_RACE", "LAW_CAT_CD"])
arrest_df_encoded.head()

,PD_CD,KY_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX_F,PERP_SEX_M,PERP_RACE_AMERICAN INDIAN/ALASKAN NATIVE,PERP_RACE_ASIAN / PACIFIC ISLANDER,PERP_RACE_BLACK,PERP_RACE_BLACK HISPANIC,PERP_RACE_UNKNOWN,PERP_RACE_WHITE,PERP_RACE_WHITE HISPANIC,LAW_CAT_CD_F,LAW_CAT_CD_I,LAW_CAT_CD_M,LAW_CAT_CD_V
0,759.0,359.0,K,79,24,0,1,0,0,1,0,0,0,0,0,0,1,0
1,101.0,344.0,M,6,24,0,1,0,0,0,0,0,1,0,0,0,1,0
2,905.0,347.0,K,72,44,0,1,0,0,0,0,0,0,1,0,0,1,0
3,782.0,236.0,Q,115,44,0,1,0,0,0,1,0,0,0,0,0,1,0
4,258.0,351.0,Q,104,24,0,1,0,0,0,0,0,1,0,0,0,1,0


In [9]:
# Define the features set.
X = arrest_df_encoded.copy()
X = X.drop("ARREST_BORO", axis=1)

# Define the target set.
y = arrest_df_encoded["ARREST_BORO"].values

X_train, X_test, y_train, y_test = train_test_split( X,y, random_state=42)

In [10]:
print(X_train.shape)
print(y_train.shape)

(2141355, 17)
(2141355,)


In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train )

RandomForestClassifier(n_estimators=200)

In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
y_preds = model.predict(X_test)

In [15]:
print(y_preds)

['K' 'B' 'M' ... 'K' 'Q' 'M']


In [16]:
# Confusion Matrix
confusion_matrix(y_test, y_preds)

array([[158980,      0,      1,      0,      0],
       [     1, 198472,      0,      0,      0],
       [     1,      0, 190482,     96,      0],
       [     0,      1,      6, 139367,      0],
       [     0,      0,      0,      0,  26379]])

In [17]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2141355, 17)
(713786, 17)
(2141355,)
(713786,)


In [18]:
# Splitting into Train and Test sets into an 80/20 split.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.80)

In [19]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2284112, 17)
(571029, 17)
(2284112,)
(571029,)


In [20]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [22]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [23]:
print(predictions)

['K' 'B' 'S' ... 'Q' 'M' 'K']


In [24]:
from sklearn.metrics import classification_report
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           B       1.00      1.00      1.00    126942\n           K       1.00      1.00      1.00    158989\n           M       1.00      1.00      1.00    152060\n           Q       1.00      1.00      1.00    111497\n           S       1.00      1.00      1.00     21541\n\n    accuracy                           1.00    571029\n   macro avg       1.00      1.00      1.00    571029\nweighted avg       1.00      1.00      1.00    571029\n'

In [25]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
# Create a DataFrame from the confusion matrix.
cm_df=pd.DataFrame(
      cm, index = ['B','K','M','Q','S'], columns = ['B','K','M','Q','S'])
cm_df

,B,K,M,Q,S
B,126941,1,0,0,0
K,0,158988,0,1,0
M,0,0,151981,79,0
Q,1,0,16,111480,0
S,0,0,0,0,21541


In [26]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [27]:
#(True Positives (TP) + True Negatives (TN)) / Total = (50 + 19)/125 = 0.552

SyntaxError: ignored

In [28]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,B,K,M,Q,S
B,126941,1,0,0,0
K,0,158988,0,1,0
M,0,0,151981,79,0
Q,1,0,16,111480,0
S,0,0,0,0,21541


Accuracy Score : 0.9998283799947113
Classification Report
              precision    recall  f1-score   support

           B       1.00      1.00      1.00    126942
           K       1.00      1.00      1.00    158989
           M       1.00      1.00      1.00    152060
           Q       1.00      1.00      1.00    111497
           S       1.00      1.00      1.00     21541

    accuracy                           1.00    571029
   macro avg       1.00      1.00      1.00    571029
weighted avg       1.00      1.00      1.00    571029

